In [1]:
# Load libraries

# # Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.ticker as ticker 
import matplotlib.patches as patches
import matplotlib as matplotlib
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Analysis
import numpy as np 
import xarray as xr
import datetime
from   datetime import date, timedelta
import pandas as pd 
import scipy.stats as stats
## Need to use metPy conda env
import metpy.calc as mpc
from metpy.units import units
import pickle
import glob

# Import Ngl with pyn_env active 
import Ngl


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:

# Grabbed from Brian M. to use time midpoints, not end periods
def cesm_correct_time(ds):
    """Given a Dataset, check for time_bnds,
       and use avg(time_bnds) to replace the time coordinate.
       Purpose is to center the timestamp on the averaging inverval.   
       NOTE: ds should have been loaded using `decode_times=False`
    """
    assert 'time_bnds' in ds
    assert 'time' in ds
    correct_time_values = ds['time_bnds'].mean(dim='nbnd')
    # copy any metadata:
    correct_time_values.attrs = ds['time'].attrs
    ds = ds.assign_coords({"time": correct_time_values})
    ds = xr.decode_cf(ds)  # decode to datetime objects
    return ds


In [4]:
'''
Define a function to get the height of the PBL as the level with maximum d(var)/dz. 
Inputs:  A dataset with CAM output ('DS') and the variable to differentiate ('pbl_var')
Outputs: An array with boundary layer depth
'''
def PBLHasMaxDZ(DS, pbl_var): 
    # Convert HMGcamDS_all to height (nabbed from Rich's script)
    p0 = DS['P0'].values[0]
    
    plevm = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
    plevm.attrs['units'] = "Pa"

    # Height with standard atmosphere
    zlevm      = plevm
    zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
    zlevm      = plevm.copy(deep=True)
    zlevm[:,:] = zlevm_vals
    
    pvar        = DS[pbl_var].isel(lat=0,lon=0)
    pvar['lev'] = zlevm[0,:].values
    dvardz      = pvar.differentiate("lev") # Find field gradient wrt HEIGHT!

    dvardz.loc[:,200:]   = 0.  # Restrict to a specificheight region
    dvardz.loc[:,:3000.] = 0

    nT = np.shape(dvardz)[0]
    PBLdepth = np.full([nT], np.nan)

    for iT in range(nT):
        iLevs  = np.where((zlevm[iT,:]>=200) & (zlevm[iT,:]<=3000))[0]
        maxLev = np.where(dvardz[iT,iLevs]==np.nanmax(dvardz[iT,iLevs]))[0]
        PBLdepth[iT] = zlevm[iT,iLevs[maxLev[0]]]
    
    return PBLdepth

In [5]:
def interpolateToPressure_v2(DS, varName, pressGoals):
#     nCases = len(DSin.case.values)
#     nTimes = len(DSin.time.values)
    
#     saveOut = np.full([nTimes,len(pressGoals),1,1], np.nan)

    ## For the larger arrays, need to operate case-by-case; input to vinth2p can only be 3 or 4 dimensions. 
#     for iCase in range(nCases): 
#     DS = DSin

    p0mb = DS.P0.values[0]/100        # mb

    # Pull out hya/hyb profiles 
    hyam = DS.hyam.values[0,:]
    hybm = DS.hybm.values[0,:]
    hyai = DS.hyai.values[0,:]
    hybi = DS.hybi.values[0,:]

    # Surface pressure with time dimension
    PS   = DS.PS.values              # Pa 

    # Converting variables: 
    if np.shape(DS[varName].values)[1]==len(DS.ilev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyai,hybi,pressGoals,PS,1,p0mb,1,True)
    elif np.shape(DS[varName].values)[1]==len(DS.lev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyam,hybm,pressGoals,PS,1,p0mb,1,True)

    saveOut = varInterp
    
    return saveOut

## Read in data

In [6]:
dataDir = '/glade/scratch/mdfowler/archive/'

yearStrings  = np.asarray(['2015','2016','2017','2018'])

fileCount = 0
for iYr in range(len(yearStrings)): 
    # caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HMG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc_useTheta_'+yearStrings[iYr]+'jja_relaxTlongTau_freeSfcT_pert01'
    caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc_useTheta_'+yearStrings[iYr]+'jja_relaxTlongTau_freeSfcT'

    ## Atm files 
    fileDir  = dataDir+caseName+'/atm/hist/' 
    lndDir   = dataDir+caseName+'/lnd/hist/' 
    
    listFiles = np.sort(glob.glob(fileDir+'*h1*'))
    h0Files   = np.sort(glob.glob(lndDir+'*h0*'))
    h1Files   = np.sort(glob.glob(lndDir+'*h1*'))
    
    for iFile in range(len(listFiles)): 
        with xr.open_dataset(listFiles[iFile], decode_times=False) as camDS:
            camDS = cesm_correct_time(camDS)
            camDS['time'] = camDS.indexes['time'].to_datetimeindex() 
            
        with xr.open_dataset(h0Files[iFile], decode_times=True) as clmDS: 
            clmDS['time'] = camDS['time']

        with xr.open_dataset(h1Files[iFile], decode_times=True) as clmPatchDS: 
            clmPatchDS['time'] = camDS['time']

        # Discard the first two days if iFile == 0  
        if iFile==0:
            iTimeStart   = np.where(camDS.time.values >= (camDS.time.values[0] + np.timedelta64(2,'D')))[0]
            timeArr      = np.arange(iTimeStart[0], len(camDS.time.values))

            camDS        = camDS.isel(time=timeArr)
            clmDS        = clmDS.isel(time=timeArr)
            clmPatchDS   = clmPatchDS.isel(time=timeArr)

        if fileCount==0:
            camDS_all      = camDS
            clmDS_all      = clmDS
            clmPatchDS_all = clmPatchDS
        else: 
            camDS_all      = xr.concat([camDS_all,     camDS],      dim='time')
            clmDS_all      = xr.concat([clmDS_all,     clmDS],      dim='time', data_vars='minimal')
            clmPatchDS_all = xr.concat([clmPatchDS_all,clmPatchDS], dim='time', data_vars='minimal')
            
        fileCount = fileCount+1
        
        print('Done with file %i of %i '% (fileCount,len(yearStrings)*len(listFiles)))
            
    
 

Done with file 1 of 44 
Done with file 2 of 44 
Done with file 3 of 44 
Done with file 4 of 44 
Done with file 5 of 44 
Done with file 6 of 44 
Done with file 7 of 44 
Done with file 8 of 44 
Done with file 9 of 44 
Done with file 10 of 44 
Done with file 11 of 44 
Done with file 12 of 44 
Done with file 13 of 44 
Done with file 14 of 44 
Done with file 15 of 44 
Done with file 16 of 44 
Done with file 17 of 44 
Done with file 18 of 44 
Done with file 19 of 44 
Done with file 20 of 44 
Done with file 21 of 44 
Done with file 22 of 44 
Done with file 23 of 44 
Done with file 24 of 44 
Done with file 25 of 44 
Done with file 26 of 44 
Done with file 27 of 44 
Done with file 28 of 44 
Done with file 29 of 44 
Done with file 30 of 44 
Done with file 31 of 44 
Done with file 32 of 44 
Done with file 33 of 44 
Done with file 34 of 44 
Done with file 35 of 44 
Done with file 36 of 44 
Done with file 37 of 44 
Done with file 38 of 44 
Done with file 39 of 44 
Done with file 40 of 44 
Done with

In [7]:
# dataDir = '/glade/scratch/mdfowler/archive/'

# yearStrings  = np.asarray(['2015','2016','2017','2018'])

# fileCount = 0
# for iYr in range(len(yearStrings)): 
#     caseName = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTG_betterFinidat4_MoreMomentsNoReorder_5mDt_realSfc_useTheta_'+yearStrings[iYr]+'jja_relaxTlongTau_freeSfcT_pert07'
    
#     fileDir   = dataDir+caseName+'/atm/hist/' 
#     extraFile = glob.glob(fileDir+'*h0*ExtraVars.nc')
    
#     with xr.open_dataset(extraFile[0], decode_times=False) as extraDS:
#         extraDS = cesm_correct_time(extraDS)
#         extraDS['time'] = extraDS.indexes['time'].to_datetimeindex() 

#     # Discard the first two days if iFile == 0  
#     iTimeStart   = np.where(extraDS.time.values >= (extraDS.time.values[0] + np.timedelta64(2,'D')))[0]
#     timeArr      = np.arange(iTimeStart[0], len(extraDS.time.values))

#     extraDS      = extraDS.isel(time=timeArr)

#     if fileCount==0:
#         extraDS_all = extraDS

#     else: 
#         extraDS_all = xr.concat([extraDS_all,extraDS], dim='time')

#     fileCount = fileCount+1
#     print('Done with file %i of %i '% (fileCount,len(yearStrings)*len(extraFile)))

    
# # Merge into larger dataset
# camDS_all = xr.merge([camDS_all, extraDS_all])

Done with file 1 of 4 
Done with file 2 of 4 
Done with file 3 of 4 
Done with file 4 of 4 


## Now do the processing


In [8]:
# ## Add evaporative fraction to DS 
# ds_EF = camDS_all.LHFLX.values / (camDS_all.LHFLX.values + camDS_all.SHFLX.values)
# camDS_all['EvapFraction'] = (('time'), np.squeeze(ds_EF))

# ## Define the actual vertical velocity skew, not just the third order moment 
# skw_W = camDS_all.WP3_CLUBB.values / ((camDS_all.WP2_CLUBB.values)**1.5)
# camDS_all['Skw_W'] = (('time','ilev'), np.squeeze(skw_W))


# ## Steps to get PBL 

# ## Get *potential* temperature, not just T 

# # So first, get actual pressures 
# p0       = camDS_all['P0'].values[0]
# plevm    = camDS_all['hyam']*p0 + camDS_all['hybm']*camDS_all['PS'].isel(lat=0,lon=0) # Mid level
# plevm.attrs['units'] = "Pa"

# theta = np.asarray(mpc.potential_temperature(plevm * units.pascals, camDS_all['T'] * units.kelvin))

# # Add to existing DS
# camDS_all['theta'] = (('time','lev','lat','lon'), theta)

# # Height with standard atmosphere
# zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
# zlevm      = plevm.copy(deep=True)
# zlevm[:,:] = zlevm_vals

# # Now compute the BL depth and save it to the larger CAM datasets 
# PBLdepth = PBLHasMaxDZ(camDS_all, 'theta')
# print('Done computing PBL depth with theta')

# # Add above to each dataset
# camDS_all['PBLdepth']    = (('time'), PBLdepth)



Done computing PBL depth with theta


**Convert to local time**

In [7]:
## Convert to local times...
# camDS_localReal         = camDS_all.copy(deep=True)
clmDS_localReal           = clmDS_all.copy(deep=True)
clmPatchDS_localReal      = clmPatchDS_all.copy(deep=True)

# Confirmed that all the times are identical, so using the same local time arrays
localTimes = camDS_all['time'].values - np.timedelta64(5,'h')

# Replace time dimension with local time
# camDS_localReal    = camDS_localReal.assign_coords({"time": localTimes})
clmDS_localReal      = clmDS_localReal.assign_coords({"time": localTimes})
clmPatchDS_localReal = clmPatchDS_localReal.assign_coords({"time": localTimes})

# print('First four times in UTC:\n',   camDS_all.time.values[0:5])
# print('First four times in local:\n', camDS_localReal.time.values[0:5])
print('First four times in UTC:\n',   clmDS_all.time.values[0:5])
print('First four times in local:\n', clmDS_localReal.time.values[0:5])


First four times in UTC:
 ['2015-06-02T23:32:15.000000000' '2015-06-02T23:37:15.000000000'
 '2015-06-02T23:42:15.000000000' '2015-06-02T23:47:15.000000000'
 '2015-06-02T23:52:15.000000000']
First four times in local:
 ['2015-06-02T18:32:15.000000000' '2015-06-02T18:37:15.000000000'
 '2015-06-02T18:42:15.000000000' '2015-06-02T18:47:15.000000000'
 '2015-06-02T18:52:15.000000000']


In [8]:

saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/'
pickle.dump( clmDS_localReal,      open( saveDir+"realSfc_HTGclm_rtpthlponly_relaxTlongTau_freeSfcT.p", "wb" ) )
pickle.dump( clmPatchDS_localReal, open( saveDir+"realSfc_HTGclmPatch_rtpthlponly_relaxTlongTau_freeSfcT.p", "wb" ) )
print('Saved interp CLM files to pickle')


Saved interp CLM files to pickle


**More processing**

In [10]:
## Belated realization that the heights computed are above *sea level* not above ground level. 
#    Need to subtract elevation. 
nateFile = '/glade/work/mdfowler/CLASP/clasp-htg.bdate.nc'
nateDS = xr.open_dataset(nateFile, decode_times=True)
elevation = nateDS.alt.values

camDS_localReal['PBLdepth']   = camDS_localReal['PBLdepth']  - elevation[0][0] 


In [11]:
## Add in TKE 
camDS_localReal['TKE']  = (('time','ilev','lat,','lon'),
                   0.5*(camDS_localReal['UP2_CLUBB']+camDS_localReal['VP2_CLUBB']+camDS_localReal['WP2_CLUBB'])) 

camDS_localReal['TKE'].attrs['units']   = 'm2/s2'
camDS_localReal['TKE'].attrs['long_name']   = 'Turbulent Kinetic Energy'


In [12]:
## Add in wind speed 
camDS_localReal['WindMagnitude']  = (('time','lev','lat,','lon'),
                                np.sqrt((camDS_localReal.U.values**2.0) + (camDS_localReal.V.values**2.0)) )

camDS_localReal['WindMagnitude'].attrs['units']   = 'm/s'
camDS_localReal['WindMagnitude'].attrs['long_name']   = 'Wind speed'


**Interpolate to standard pressure levels**

In [13]:
## Decide on levels to interpoalte to and add to larger arrays
pnew64 = np.arange(200.0,980.0,10.0) 

camDS_localReal = camDS_localReal.assign_coords({"levInterp": pnew64})

varSels = np.asarray(['THLP2_CLUBB','RTP2_CLUBB','RTPTHLP_CLUBB','WPRTP_CLUBB','WPTHLP_CLUBB','WP2_CLUBB','UP2_CLUBB',
                      'VP2_CLUBB','TKE','U','V','T','Q','OMEGA','RVMTEND_CLUBB','STEND_CLUBB','CLOUD',
                      'UPWP_CLUBB','VPWP_CLUBB','WP2RTP_CLUBB','THETAL','QRL','QRS','DCQ','WindMagnitude'])


for iVar in range(len(varSels)): 
    varUnits = camDS_localReal[varSels[iVar]].units
    varName  = camDS_localReal[varSels[iVar]].long_name
    
    # Interpolate variables and add to larger arrays 
    interpVar_real = interpolateToPressure_v2(camDS_localReal,     varSels[iVar], pnew64)
    
    camDS_localReal[varSels[iVar]+'_interp']  = (('time','levInterp','lat','lon'), interpVar_real)
     
    ## Assign attibutes 
    camDS_localReal[varSels[iVar]+'_interp'].attrs['units']     = varUnits
    camDS_localReal[varSels[iVar]+'_interp'].attrs['long_name'] = varName

    
    print('Done with variable %i of %i' % (iVar, len(varSels)))
    

Done with variable 0 of 25
Done with variable 1 of 25
Done with variable 2 of 25
Done with variable 3 of 25
Done with variable 4 of 25
Done with variable 5 of 25
Done with variable 6 of 25
Done with variable 7 of 25
Done with variable 8 of 25
Done with variable 9 of 25
Done with variable 10 of 25
Done with variable 11 of 25
Done with variable 12 of 25
Done with variable 13 of 25
Done with variable 14 of 25
Done with variable 15 of 25
Done with variable 16 of 25
Done with variable 17 of 25
Done with variable 18 of 25
Done with variable 19 of 25
Done with variable 20 of 25
Done with variable 21 of 25
Done with variable 22 of 25
Done with variable 23 of 25
Done with variable 24 of 25


In [14]:
saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/'
pickle.dump( camDS_localReal,   open( saveDir+"realSfc_HTG_rtpthlponly_relaxTlongTau_freeSfcT_pert07.p", "wb" ) )
print('Saved interp CAM files to pickle')


Saved interp CAM files to pickle
